In [2]:
from huggingface_hub import notebook_login
from datasets import load_dataset

notebook_login()

In [ ]:
##requirements to run in terminal
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

In [3]:
##read corpus data
data_spanish=load_dataset("spanish_billion_words",split="train")

Reusing dataset spanish_billion_words (/home/jhonparra/.cache/huggingface/datasets/spanish_billion_words/corpus/1.1.0/8ba50a854d61199f7d36b4c3f598589a2f8b493a2644b88ce80adb2cebcbc107)


In [4]:
from utils import clean_batch,homologate_accents

column_text="text"

def extract_all_chars(batch):
  all_text = " ".join(batch[column_text])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [9]:
##preprocessing data

data_spanish=data_spanish.map(clean_batch,fn_kwargs={"text_column":column_text})
data_spanish=data_spanish.map(homologate_accents,fn_kwargs={"text_column":column_text})

  0%|          | 0/46925295 [00:00<?, ?ex/s]

  0%|          | 0/46925295 [00:00<?, ?ex/s]

### Push dataset to hub

Lets generate a checkpoint for processed data

In [ ]:

target_lang="es" 
data_spanish.push_to_hub(f"spanish_billion_words_clean", split="train")


### Checkpoint

In [1]:
from datasets import load_dataset
dataset_processed = load_dataset(f"jhonparra18/spanish_billion_words_clean", split="train")

Using custom data configuration jhonparra18--spanish_billion_words_clean-afd4c7aa3941f2eb
Reusing dataset parquet (/home/jhon.parra/.cache/huggingface/datasets/parquet/jhonparra18--spanish_billion_words_clean-afd4c7aa3941f2eb/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


In [2]:
##given memory limitations, a sample dataset must be used
dataset_processed_sample=dataset_processed.train_test_split(train_size=0.2)["train"]
print(dataset_processed_sample.shape)

(4692529, 1)


In [3]:
with open("text.txt", "w") as file:
  file.write(" ".join(dataset_processed_sample["text"]))

In [1]:
##after generating .arpa file
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor,Wav2Vec2Processor,Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")

model = Wav2Vec2ForCTC.from_pretrained("jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")

In [4]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")

In [5]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [6]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

     |████████████████████████████████| 541 kB 44 kB/s eta 0:00:011
     |████████████████████████████████| 43 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 376 kB 5.8 MB/s eta 0:00:01
  Created wheel for kenlm: filename=kenlm-0.0.0-cp38-cp38-linux_x86_64.whl size=2977431 sha256=568a80e27bb5be960de83d2231c08e4c8ec40cc5eaf0d8e3df9686209643b0f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-fo8rpfhr/wheels/ff/08/4e/a3ddc0e786e0f3c1fcd2e7a82c4324c02fc3ae2638471406d2
  Created wheel for pygtrie: filename=pygtrie-2.4.2-py3-none-any.whl size=19062 sha256=f26467546cd05e7f9b8ffc7555029c7a372022f58b86e4aef7dc7d206e9f4f31
  Stored in directory: /home/jhon.parra/.cache/pip/wheels/31/03/7b/f685b394a937bc97d2d40908d45aa31f3d9473bca6e9019153
Successfully built kenlm pygtrie


In [8]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
No known unigrams provided, decoding results might be a lot worse.


In [9]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [13]:
##install git-lfs
!sudo apt-get install git-lfs tree

In [11]:
##clone repo from hf
from huggingface_hub import Repository

repo = Repository(local_dir="hf-asr-comp", clone_from="jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom")


In [10]:
##save model
processor_with_lm.save_pretrained("hf-asr-comp")

In [ ]:
##in case 5gram file is too big, convert it to binary using the following in the command line
!kenlm/build/bin/build_binary hf-asr-comp/language_model/5gram_correct.arpa hf-asr-comp/language_model/5gram.bin
##then delete old file to avoid it from being added to the repo
!rm hf-asr-comp/language_model/5gram_correct.arpa && tree -h hf-asr-comp/

In [12]:
repo.push_to_hub(commit_message="LM added using 5-gram model")

Adding files tracked by Git LFS: ['language_model/unigrams.txt']. This may take a bit of time if the files are large.
Upload file language_model/5gram.bin: 14.0GB [24:08, 12.3MB/s]                            To https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom
   ef92bd3..90f6ce6  main -> main

Upload file language_model/5gram.bin: 100%|██████████| 9.68G/9.68G [24:10<00:00, 7.17MB/s]
Upload file language_model/unigrams.txt: 100%|██████████| 12.5M/12.5M [24:10<00:00, 9.03kB/s]


'https://huggingface.co/jhonparra18/wav2vec2-large-xls-r-300m-spanish-custom/commit/90f6ce68df655dd2ebff32d542c733506c2dad13'